In [ ]:
import numpy as np
import scipy.signal as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
sys.path.append('../')
import utils as ut

In [ ]:
folder = '/Users/Jan/Dropbox/Master/LR_Kuehn/data/dystonia_rest/for_python/good_theta/'
file = 'spmeeg_13.mat'
d = ut.load_data_spm(file, folder)

In [ ]:
lfp = d['data'][1]
fs = d['fsample'][0][0]

## Design a filter with firls method, explicitly

In [ ]:
# the filter order, should 3x the cycle length of the smallest frequency in the desired band 
nyq = fs / 2.
band = [4., 70.]
# get the number of samples per ms
samples_per_ms = fs / 1000.
# get the cycle length in samples 
cycle_length = int(1000 / band[0] * samples_per_ms)
numtaps = 3 * cycle_length
# make numtaps even 
if not numtaps % 2: 
    numtaps += 1

In [ ]:
# set the filter parameters
start = band[0] - np.floor(0.1 * (band[1] - band[0])) 
if start < 1: 
    start = 1.
stop = band[1] + np.ceil(0.1 * (band[1] - band[0]))
bands = (0, start, band[0], band[1], stop, nyq)
# set the desired filter response 
desired = (0, 0, 1, 1, 0, 0)
# design the filter 
coefs = sp.firls(numtaps, bands, desired, nyq=nyq)

## Design it with the window method 

In [ ]:
# design the filter 
coefs_win = sp.firwin(numtaps=numtaps, cutoff=(band[0], band[1]), window='hamming', nyq=nyq, pass_zero=False)
# get the frequency response 
freq_win, response_win = sp.freqz(coefs_win)

## Design IIR filter with butter

In [ ]:
wn = band / nyq
b, a = sp.butter(Wn=wn, N=4, btype='bandpass')
freq_butter, response_butter = sp.freqz(b, a)

In [ ]:
# get the frequency response 
freq, response = sp.freqz(coefs)
# plot 
plt.figure(figsize=(15, 10))
upto = int(band[1] + 20)
plt.semilogy((nyq*freq/(np.pi))[:upto], np.abs(response)[:upto], label='firs')
plt.semilogy((nyq*freq_win/(np.pi))[:upto], np.abs(response_win)[:upto], label='firwin')
plt.semilogy((nyq*freq_butter/(np.pi))[:upto], np.abs(response_butter)[:upto], label='butter')
plt.semilogy(bands, desired)
plt.xlim([0, upto + 10])
plt.legend();

## Check goodness of filters 

In [ ]:
desired = np.zeros_like(response)
f = nyq*freq/(np.pi)
mask = ut.get_array_mask(f >= band[0], f <= band[1])
desired[mask] = 1 

In [ ]:
ss = np.sum((desired - abs(response))**2)
ss_win = np.sum((desired - abs(response_win))**2)
ss_butter = np.sum((desired - abs(response_butter))**2)
print(ss)
print(ss_win)
print(ss_butter)

## Apply the filters and compare results

In [ ]:
y = sp.filtfilt(coefs, 1, lfp)
y_win = sp.filtfilt(coefs_win, 1, lfp)
y_butter = sp.filtfilt(b, a, lfp)

In [ ]:
plt.figure(figsize=(15, 10))
start = 1000
stop = start + 100
plt.plot(y[start:stop], label='firs')
plt.plot(y_win[start:stop], label='firwin')
plt.plot(y_butter[start:stop], label='butter')
plt.legend()
plt.plot(lfp[start:stop])



## Conclusion
Looking at the value of the SS between the desired and the actual response it seems that the firwin method is the best choice because it automatically chooses the transition zones. 

The choice of the transition zones becomes more difficult if the lower bandpass is small and the range is large, etc. 

The butter IIR filter has a bad frequency resolution, e.g., see the green curve in the response plot and the large value in the sum of squares. But this may be due to the fact that it has an infinite impulse response? 